## 기본 예시: 프롬프트 + 모델 + 출력 파서

- 가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 
- 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [5]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [6]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [7]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [8]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

- 파이프라인 == Chain

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [10]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")

model = ChatOpenAI()

# 프롬프트 결과가 모델로 자동 전달됨
chain = prompt | model

In [11]:
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 {how} 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x16db95410>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x16dba1250>, root_client=<openai.OpenAI object at 0x16db85c50>, root_async_client=<openai.AsyncOpenAI object at 0x16db955d0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.
- 답변이 완성될때까지 기다렸다가 한 번에 호출됨

In [12]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {
    "topic": "인공지능 모델의 학습 원리",
    "how": "3줄로",
}

In [20]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델은 데이터를 입력으로 받아 가중치를 조정하고, 손실 함수를 최소화하는 방향으로 학습합니다. 학습 과정에서 모델은 입력 데이터와 정답 레이블을 비교하여 오차를 계산하고, 역전파 알고리즘을 통해 가중치를 업데이트합니다. 이러한 과정을 반복하여 모델이 데이터 패턴을 학습하고 예측을 수행할 수 있게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 34, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-426e6879-5d3c-466d-90cc-127440730c5d-0', usage_metadata={'input_tokens': 34, 'output_tokens': 148, 'total_tokens': 182, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 스트리밍을 출력하는 예시 입니다.

In [21]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델은 데이터를 입력하고 목표값과 비교하여 오차를 계산하고, 이 오차를 최소화하는 방향으로 가중치를 조정하면서 학습합니다. 이 과정을 반복하여 모델이 데이터에 대해 더 잘 일반화되도록 학습합니다. 학습이 끝나면 모델은 새로운 데이터에 대해 예측을 할 수 있습니다.

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [15]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how": "쉽게"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 패턴을 학습하고 예측하는 과정을 말합니다. 모델은 입력 데이터를 특정한 방식으로 처리하여 출력을 생성하고, 이때 발생하는 오차를 최소화하도록 학습합니다. 학습 과정은 반복적으로 이루어지며, 모델은 오차를 줄이기 위해 가중치와 편향을 조절하면서 최적의 성능을 찾아나갑니다. 이렇게 모델은 데이터를 효율적으로 학습하여 새로운 데이터에 대해 정확한 예측을 할 수 있도록 되는 것이 인공지능 모델의 핵심 원리입니다.'

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 통해 패턴을 학습하는 과정입니다. 먼저 모델은 입력 데이터를 받아들이고, 이 데이터를 특정한 방식으로 처리하여 출력을 생성합니다. 이때 모델은 정답과 비교하여 오차를 계산하고, 이 오차를 최소화하는 방향으로 가중치와 편향을 조정하면서 학습합니다. 이 과정을 반복하면 모델은 점차 데이터의 패턴을 학습하게 되고, 정확한 결과를 예측할 수 있게 됩니다. 이렇게 학습된 모델은 새로운 데이터에 대해 예측을 할 수 있게 되며, 이를 통해 다양한 문제를 해결할 수 있습니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [19]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [20]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [21]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  - Server: Hello! Welcome to our restaurant. How many people are in your party?
  - You: Hi! It's just me today.
  - Server: Great! Please follow me to your table. Here's the menu. Can I get you something to drink while you decide?
  - You: Yes, I would like a glass of water, please.
  - Server: Sure, I'll be right back with your water. Take your time with the menu.
  - (A few minutes later)
  - Server: Here’s your water. Have you decided what you’d like to order?
  - You: Yes, I’ll have the grilled chicken with a side of vegetables, please.
  - Server: Excellent choice! Would you like any appetizers or desserts today?
  - You: No, thank you. Just the main course.
  - Server: Alright, your order will be ready shortly. Enjoy your meal!

- 한글 해석:
  - 종업원: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?
  - 당신: 안녕하세요! 오늘은 저 혼자입니다.
  - 종업원: 좋습니다! 테이블로 안내해 드릴게요. 여기 메뉴입니다. 메뉴를 보시는 동안 음료를 가져다 드릴까요?
  - 당신: 네, 물 한 잔 주세요.
  - 종업원: 네, 물 바로 가져다 드리겠습니다. 메뉴 천천히 보세요.
  - (몇 분 후)
  - 종업원: 여기 물입니다. 주문하실 

In [22]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Waiter: Good evening! Welcome to our restaurant. How many people are in your party?
  - Customer: Good evening! It's just me today.
  - Waiter: Great, please follow me to your table. Here's the menu. Can I get you something to drink to start with?
  - Customer: Yes, I'll have a glass of water, please.
  - Waiter: Certainly. Are you ready to order, or would you like a few more minutes?
  - Customer: I'm ready to order. I'll have the grilled chicken with a side of mashed potatoes.
  - Waiter: Excellent choice. Would you like any appetizers or desserts with that?
  - Customer: No, thank you. Just the main course for now.
  - Waiter: Perfect. Your order will be ready shortly. Enjoy your meal!

- 한글 해석:
  - 웨이터: 안녕하세요! 저희 레스토랑에 오신 것을 환영합니다. 몇 분이신가요?
  - 손님: 안녕하세요! 오늘은 저 혼자입니다.
  - 웨이터: 좋습니다, 저를 따라 테이블로 오세요. 여기 메뉴입니다. 시작으로 음료를 드릴까요?
  - 손님: 네, 물 한 잔 주세요.
  - 웨이터: 알겠습니다. 주문할 준비가 되셨나요? 아니면 몇 분 더 필요하신가요?
  - 손님: 주문할 준비가 됐어요. 그릴드 치킨과 매쉬드 포테이토를 주세요.
  - 웨이터: 좋은 선택이세요. 전채나 디저트를 추가하시겠어

In [23]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Customer: Hi, I'd like to order a pizza for delivery, please.
  - Employee: Sure! What size pizza would you like?
  - Customer: I'll have a large pizza.
  - Employee: Great choice. What toppings would you like on your pizza?
  - Customer: Could I get pepperoni, mushrooms, and extra cheese?
  - Employee: Absolutely. Would you like any sides or drinks with that?
  - Customer: Yes, I’ll take a side of garlic bread and a bottle of cola.
  - Employee: Perfect. Can I have your address and phone number for the delivery?
  - Customer: Sure, my address is 123 Main Street, and my phone number is 555-0123.
  - Employee: Thank you. Your total comes to $25.99. It should arrive in about 30 minutes.
  - Customer: Sounds good! Thank you very much.
  - Employee: You're welcome! Have a great day!

- 한글 해석:
  - 고객: 안녕하세요, 피자를 배달 주문하고 싶습니다.
  - 직원: 네! 어떤 사이즈의 피자를 원하시나요?
  - 고객: 큰 사이즈로 주세요.
  - 직원: 좋은 선택입니다. 피자에 어떤 토핑을 원하시나요?
  - 고객: 페퍼로니, 버섯, 그리고 치즈 추가해 주세요.
  - 직원: 물론입니다. 사이드나 음료 추가하시겠어요?
  